In [1]:
import pandas as pd

In [59]:
df = pd.read_csv('../tosdr_data.csv')
# get rid of unamed column
df = df[["title", "classification"]]
df

,title,classification
0,You can delete your content from this service,good
1,A license is kept on user-generated content ev...,bad
2,This service tracks you on other websites,bad
3,You agree to comply with the law of the servic...,neutral
4,This service retains rights to your content ev...,bad
...,...,...
248,"You agree to defend, indemnify, and hold the s...",neutral
249,This service assumes no liability for any loss...,neutral
250,Information is provided about what kind of inf...,good
251,Your personal data is used for automated decis...,blocker


In [65]:
data_cleaned = df
X = df['title']
y_classification = df['classification']


In [69]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.pipeline import Pipeline as ImbPipeline  # Use imblearn's Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer()),
    ('smote', SMOTE(random_state=42)),  # Handle class imbalance
    ('clf', LogisticRegression())
])




# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Text preprocessing function
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stopwords.words('english')]
    return ' '.join(words)

# Preprocess your data
X_preprocessed = [preprocess_text(text) for text in X]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_classification, test_size=0.2, random_state=42)

# Define a pipeline with TF-IDF and Logistic Regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('smote', SMOTE(random_state=42)),  # Handle class imbalance
    ('clf', LogisticRegression())
])

# Define grid search parameters
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1, 10]
}

# Grid Search with Cross-Validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Train the model
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_

# Predict and evaluate
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Best Parameters:", best_params)
print("Accuracy:", accuracy)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jessicasanthanam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jessicasanthanam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jessicasanthanam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/pipeline.py", line 350, in _fit
    self._valid

In [54]:
# create data frame for y_pred_sev with column name 'severity'
y_pred_sev_df = pd.DataFrame(y_pred_sev, columns=['severity'])
y_pred_sev_df['severity'].value_counts()

severity
0    2379
2      61
3      21
1       3
Name: count, dtype: int64

In [48]:
import pandas as pd
new_data = pd.read_csv('../data/tosdr_data.csv')
new_clauses = new_data['title']

# Predicting on the new data
new_clauses_tfidf = vectorizer.transform(new_clauses)
new_pred_sev = severity_classifier.predict(new_clauses_tfidf)


# create new dataframe with new clauses and predicted severity
new_df = pd.DataFrame({'clause': new_clauses, 'severity': new_pred_sev})
new_df

,clause,severity
0,You can delete your content from this service,0
1,A license is kept on user-generated content ev...,0
2,This service tracks you on other websites,0
3,You agree to comply with the law of the servic...,0
4,This service retains rights to your content ev...,0
...,...,...
248,"You agree to defend, indemnify, and hold the s...",0
249,This service assumes no liability for any loss...,0
250,Information is provided about what kind of inf...,0
251,Your personal data is used for automated decis...,0


In [55]:
new_df['severity'].value_counts()

severity
0    251
2      2
Name: count, dtype: int64

In [85]:
import requests

base_url = "https://api.tosdr.org/pihole/"
response = requests.get(base_url)
sites = response.text


# from sites only get lines with ToS;DR: in them
tosdr_sites = []
for line in sites.splitlines():
    if "ToS;DR:" in line:
        tosdr_sites.append(line[10:])

print(tosdr_sites)



['https://tosdr.org/en/service/230', 'https://tosdr.org/en/service/460', 'https://tosdr.org/en/service/183', 'https://tosdr.org/en/service/186', 'https://tosdr.org/en/service/225', 'https://tosdr.org/en/service/219', 'https://tosdr.org/en/service/989', 'https://tosdr.org/en/service/2895', 'https://tosdr.org/en/service/2315', 'https://tosdr.org/en/service/1597', 'https://tosdr.org/en/service/2256', 'https://tosdr.org/en/service/165', 'https://tosdr.org/en/service/2491', 'https://tosdr.org/en/service/2543', 'https://tosdr.org/en/service/3522', 'https://tosdr.org/en/service/445', 'https://tosdr.org/en/service/226', 'https://tosdr.org/en/service/3335', 'https://tosdr.org/en/service/3362', 'https://tosdr.org/en/service/264', 'https://tosdr.org/en/service/1741', 'https://tosdr.org/en/service/3534', 'https://tosdr.org/en/service/2929', 'https://tosdr.org/en/service/1577', 'https://tosdr.org/en/service/2529', 'https://tosdr.org/en/service/2697', 'https://tosdr.org/en/service/318', 'https://tos

In [102]:
import requests
from bs4 import BeautifulSoup

url = "https://tosdr.org/en/service/460"


def get_clauses(url):
    response = requests.get(url)
    response.raise_for_status()
    clauses = []
    classifications = []

    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize a list to hold all the texts
    li_texts = []

    # Find all the <li> tags and iterate over them
    for li in soup.find_all("li", class_="list-group-item"):
        # Extract the text within each <li> tag
        class_list = li.get('class')
        if len(class_list) > 1:
            item_type = class_list[1]  # This is the class name you're interested in
            if 'danger' in item_type:
                classifications.append('blocker')
            elif 'warning' in item_type:
                classifications.append('bad')
            elif 'secondary' in item_type:
                classifications.append('neutral')
            elif 'success' in item_type:
                classifications.append('good')
            
        text = li.get_text(strip=True)
        li_texts.append(text)

    # This will print all the extracted texts from each <li> tag
    for text in li_texts:
        clauses.append(text)
    
    return clauses, classifications


['Sensitive data is collected and shared to target advertising', 'Terms may be changed any time at their discretion, without notice to you', 'Some personal data may be kept for business interests or legal obligations', 'This service tracks you on other websites', 'You are forced into binding arbitration in case of disputes', 'You waive your right to a class action.', 'Information is gathered about you through third parties', 'This service forces users into binding arbitration in the case of disputes', 'This service reserves the right to disclose your personal information without notifying you', 'You are tracked via web beacons, tracking pixels, browser fingerprinting, and/or device fingerprinting', 'Your personal data may be sold or otherwise transferred as part of a bankruptcy proceeding or other type of financial transaction', 'The service may keep a secure, anonymized record of your data for analytical purposes even after the data retention period', 'Your personal information is use

In [104]:
# for every url in tosdr_sites, get the clauses and classifications
all_clauses = []
all_classifications = []
for url in tosdr_sites:
    print(url)
    clauses, classifications = get_clauses(url)
    all_clauses.extend(clauses)
    all_classifications.extend(classifications)

# create dataframe with clauses and classifications
df = pd.DataFrame({'clause': all_clauses, 'classification': all_classifications})
df

https://tosdr.org/en/service/230
https://tosdr.org/en/service/460
https://tosdr.org/en/service/183
https://tosdr.org/en/service/186
https://tosdr.org/en/service/225
https://tosdr.org/en/service/219
https://tosdr.org/en/service/989
https://tosdr.org/en/service/2895
https://tosdr.org/en/service/2315
https://tosdr.org/en/service/1597
https://tosdr.org/en/service/2256
https://tosdr.org/en/service/165
https://tosdr.org/en/service/2491
https://tosdr.org/en/service/2543
https://tosdr.org/en/service/3522
https://tosdr.org/en/service/445
https://tosdr.org/en/service/226
https://tosdr.org/en/service/3335
https://tosdr.org/en/service/3362
https://tosdr.org/en/service/264
https://tosdr.org/en/service/1741
https://tosdr.org/en/service/3534
https://tosdr.org/en/service/2929
https://tosdr.org/en/service/1577
https://tosdr.org/en/service/2529
https://tosdr.org/en/service/2697
https://tosdr.org/en/service/318
https://tosdr.org/en/service/403
https://tosdr.org/en/service/315
https://tosdr.org/en/service

,clause,classification
0,You waive your moral rights,blocker
1,This service still tracks you even if you opte...,blocker
2,This service holds onto content that you've de...,blocker
3,You must provide your identifiable information,bad
4,This service may keep personal data after a re...,bad
...,...,...
4963,This service is only available to users over 1...,neutral
4964,Your personal data is aggregated into statistics,neutral
4965,This service requires first-party cookies,neutral
4966,"The service may change its terms at any time, ...",neutral


In [116]:
import json
ids_num = []
for i in range(1, 90):
    new_url = f"https://api.tosdr.org/service/v2/?page={i}"


    response = requests.get(new_url)
    sites = response.text

    # Parse the JSON string into a Python dictionary
    data = json.loads(sites)

    # Extract the 'services' list
    try:
        services = data['parameters']['services']

        # Get all the 'id' values from the services
        ids = [service['id'] for service in services]
        ids_num.extend(ids)
    except:
        pass

print(ids_num)


[440, 353, 2894, 3157, 162, 605, 212, 1946, 233, 327, 674, 1222, 1225, 1216, 448, 2122, 685, 615, 179, 1625, 511, 372, 486, 230, 460, 441, 262, 168, 402, 158, 2897, 306, 2136, 183, 186, 330, 3011, 225, 2896, 251, 208, 753, 791, 802, 842, 850, 1642, 879, 1948, 219, 1890, 2015, 2938, 3518, 1569, 3480, 1938, 2187, 3206, 969, 909, 259, 293, 1546, 1005, 699, 888, 877, 1429, 1424, 1459, 2017, 1616, 1750, 2257, 2068, 989, 1342, 829, 2895, 810, 284, 817, 1960, 1939, 1574, 2063, 278, 709, 2071, 3068, 713, 1811, 1492, 1738, 1724, 1716, 1539, 2898, 2939, 3272, 1509, 2937, 3012, 3159, 3516, 2456, 3438, 1879, 1479, 398, 730, 3193, 3392, 2190, 2236, 2315, 2371, 2401, 1597, 1345, 1941, 3273, 1358, 2256, 1894, 1501, 2306, 3069, 2050, 2139, 2307, 2061, 1950, 2899, 2220, 3070, 533, 1613, 2246, 681, 3160, 622, 611, 1958, 2247, 717, 263, 356, 2022, 2000, 2638, 2416, 2726, 2901, 2433, 2888, 2641, 2221, 2470, 1527, 2400, 2623, 2700, 1430, 2393, 165, 3013, 2892, 1700, 2900, 2902, 3192, 2942, 3014, 3084, 593,

In [122]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
all_clauses = []
all_classifications = []
for id in tqdm(ids_num):
    url = f"https://tosdr.org/en/service/{id}"
    clauses, classifications = get_clauses(url)
    all_clauses.extend(clauses)
    all_classifications.extend(classifications)

df = pd.DataFrame({'clause': all_clauses, 'classification': all_classifications})
df



 36%|███▌      | 1546/4300 [39:59<1:07:59,  1.48s/it]